In [8]:
import numba
import numpy as np
import pandas as pd
from numba import cuda,njit,prange

In [9]:
import os,platform,sys
print("""Python version: %s 
platform: %s 
system: %s 
Numba version: %s""" % 
      (sys.version.split('\n'),platform.machine(),platform.platform(),numba.__version__))

Python version: ['3.6.3 |Anaconda, Inc.| (default, Oct 15 2017, 03:27:45) [MSC v.1900 64 bit (AMD64)]'] 
platform: AMD64 
system: Windows-10-10.0.14393-SP0 
Numba version: 0.40.0rc1


In [3]:
@njit(fastmath=True)
def emptyListOfInts():
    return [i for i in range(0)]
@njit(fastmath=True)
def BinByUniqueValues(array,l,r,m,mask):        
    groupedIndices=[emptyListOfInts() for k in range(m)]
    if len(mask)>0:
        i=l
        while i<r:        
            ind=mask[i]
            groupedIndices[array[ind]].append(ind)
            i+=1
    else:
        i=l
        while i<r:
            groupedIndices[array[i]].append(i)
            i+=1
    return groupedIndices
@njit(fastmath=True)
def counting_argsort_list(array,maxval,mask=np.array([],np.int32)):        
    m=maxval+1    
    
    #Allocate output array
    if len(mask)>0:
        arrayLen=len(mask)
    else:
        arrayLen=len(array)        
    argsorted=np.empty(arrayLen,np.int32)    
    
    #Group indices of same values
    groupedIndices=BinByUniqueValues(array,0,arrayLen,m,mask)
    
    position = 0
    for k in range(m):
        if len(groupedIndices[k])>0:
            for index in groupedIndices[k]:
                argsorted[position] = index
                position+= 1            
    return argsorted
@njit(fastmath=True,parallel=True)
def counting_argsort_list_threaded(array,maxval,mask=np.array([],np.int32)):        
    m=maxval+1    
    
    #Allocate output array
    if len(mask)>0:
        arrayLen=len(mask)
    else:
        arrayLen=len(array)        
    argsorted=np.empty(arrayLen,np.int32)    
    
    #Group indices of same values
    groups=[]
    effectiveSize=int(m*3)        
    if arrayLen<=effectiveSize: 
        groups.append(BinByUniqueValues(array,0,arrayLen,m,mask))
    else:
        nThreads=min(max(arrayLen//effectiveSize,1),8)            
        effectiveSize=arrayLen//nThreads
        #print("nThreads=",nThreads)
        for k in prange(nThreads):
            lBound=effectiveSize*k;rBound=effectiveSize*(k+1)
            if k==nThreads-1:
                rBound=arrayLen
            groups.append(BinByUniqueValues(array,lBound,rBound,m,mask))
    position = 0
    for k in range(m):
        for groupedIndices in groups:
            if len(groupedIndices)>0:
                if len(groupedIndices[k])>0:
                    for index in groupedIndices[k]:
                        argsorted[position] = index
                        position+= 1   
    return argsorted

In [4]:
data=np.array([1, 4, 7, 2, 1, 3, 2, 1, 4, 7, 2, 1, 3, 2, 3, 2, 1]*5)

print((data[np.argsort(data)]==data[counting_argsort_list(data,7)]).all())
print((data[np.argsort(data)]==data[counting_argsort_list_threaded(data,7)]).all())

True
Parallel for-loop #0 is produced from pattern '('prange', 'user')' at <ipython-input-3-84432365d5c1> (60)
After fusion, function counting_argsort_list_threaded has 1 parallel for-loop(s) #{0}.
True


In [5]:
data=np.random.randint(1,100,50000)

In [6]:
%timeit np.argsort(data)

2.32 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit counting_argsort_list(data,100)

1.49 ms ± 3.07 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
%timeit counting_argsort_list_threaded(data,100)